# Module 2, Task 4: Train and Evaluate a Keras-Based Classifier

**Objective:** Build, compile, train, and evaluate a Convolutional Neural Network (CNN) for image classification using TensorFlow and Keras.

In [ ]:
# Install necessary libraries
!pip install tensorflow tensorflow-datasets matplotlib scikit-learn

### Setup and Data Pipeline
We'll reuse the efficient data pipeline we built in the previous Keras notebook (M1_02). This demonstrates the modularity of the `tf.data` API.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns

# --- Data Loading and Preprocessing (from M1_02) ---
(ds_train, ds_validation), ds_info = tfds.load(
    'eurosat/rgb', split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True, as_supervised=True, with_info=True)

NUM_CLASSES = ds_info.features['label'].num_classes
IMG_SIZE = 128
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE

def process_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return tf.cast(image, tf.float32) / 255.0, label

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
], name="data_augmentation")

def configure_dataset(ds, shuffle=False):
    ds = ds.map(process_image, num_parallel_calls=AUTOTUNE)
    ds = ds.cache()
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(BATCH_SIZE)
    if shuffle:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = configure_dataset(ds_train, shuffle=True)
validation_ds = configure_dataset(ds_validation)
print("Data pipelines are ready.")

### Building the CNN Model

We'll create a simple but effective CNN architecture using the Keras Sequential API. It consists of several convolutional blocks followed by a dense classifier head.

**Convolutional Block:** `Conv2D` -> `ReLU` Activation -> `MaxPooling2D`

In [ ]:
def build_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        # Conv Block 1
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        # Conv Block 2
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        # Conv Block 3
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        # Classifier Head
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax') # Softmax for multi-class classification
    ])
    return model

model = build_model((IMG_SIZE, IMG_SIZE, 3), NUM_CLASSES)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy', # Use for integer labels
    metrics=['accuracy']
)

model.summary()

### Training the Model
Now we can train our compiled model using the `fit` method, passing our prepared `tf.data.Dataset` objects.

In [ ]:
EPOCHS = 15 # A smaller number for quick demonstration

history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=EPOCHS,
    verbose=1
)

### Evaluating the Model

After training, we must evaluate the model's performance on the validation set. We'll visualize the training history and generate a detailed classification report.

In [ ]:
# Plotting training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Detailed Evaluation
y_true = []
y_pred_probs = []

for images, labels in validation_ds:
    y_true.extend(labels.numpy())
    y_pred_probs.extend(model.predict(images, verbose=0))

y_true = np.array(y_true)
y_pred = np.argmax(y_pred_probs, axis=1)
class_names = ds_info.features['label'].names

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

### Conclusion

We have successfully built, trained, and evaluated a Keras-based CNN classifier. The process is streamlined thanks to the high-level `model.fit()` API and the efficient `tf.data` pipeline. The evaluation metrics and visualizations give us a clear picture of the model's performance on the land classification task.